# Testing Coframe API with Flask Server

This notebook demonstrates how to interact with the Coframe API through the Flask server interface.

In [ ]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# Base URL for API
BASE_URL = 'http://localhost:5000/api'

# Helper function to pretty print JSON
def pretty_print(obj):
    if isinstance(obj, dict) or isinstance(obj, list):
        display(HTML(f"<pre>{json.dumps(obj, indent=2)}</pre>"))
    else:
        print(obj)

## Authentication

First, let's authenticate with the API to get a token.

In [ ]:
def login(username, password):
    response = requests.post(f"{BASE_URL}/auth/login", json={
        'username': username,
        'password': password
    })

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Login failed: {response.status_code}")
        return response.json()

# Login with credentials
auth_result = login('mrossi', 'hashed_password_here')
pretty_print(auth_result)

In [ ]:
# Extract token from auth_result
token = auth_result.get('token')

# Setup headers for authenticated requests
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

## Basic CRUD Operations

Now let's test some basic CRUD operations on the database.

### Retrieve All Users

In [ ]:
def get_all(table, params=None):
    response = requests.get(f"{BASE_URL}/db/{table}", headers=headers, params=params)
    return response.json()

# Get all users
users = get_all('User')
pretty_print(users)

### Get User by ID

In [ ]:
def get_one(table, id):
    response = requests.get(f"{BASE_URL}/db/{table}/{id}", headers=headers)
    return response.json()

# Get user with ID=1
user = get_one('User', 1)
pretty_print(user)

### Create a New Book

In [ ]:
def create(table, data):
    response = requests.post(f"{BASE_URL}/db/{table}", headers=headers, json=data)
    return response.json()

# Create a new book
import datetime

new_book = {
    'title': 'Python for Data Science',
    'isbn': '9781234567890',
    'publication_date': datetime.date.today().isoformat(),
    'price': 29.99,
    'status': 'A'
}

created_book = create('Book', new_book)
pretty_print(created_book)

### Update the Book

In [ ]:
def update(table, id, data):
    response = requests.put(f"{BASE_URL}/db/{table}/{id}", headers=headers, json=data)
    return response.json()

# Update the book's price
if 'data' in created_book and 'id' in created_book['data']:
    book_id = created_book['data']['id']
    updated_book = update('Book', book_id, {'price': 34.99})
    pretty_print(updated_book)

### Delete the Book

In [ ]:
def delete(table, id):
    response = requests.delete(f"{BASE_URL}/db/{table}/{id}", headers=headers)
    return response.json()

# Delete the book
if 'data' in created_book and 'id' in created_book['data']:
    book_id = created_book['data']['id']
    deleted = delete('Book', book_id)
    pretty_print(deleted)

## Advanced Queries

Let's test the dynamic query builder functionality.

In [ ]:
def execute_query(query, format='tuples'):
    response = requests.post(f"{BASE_URL}/query", headers=headers, json={
        'query': query,
        'format': format
    })
    return response.json()

# Query to get books with authors
books_query = {
    "from": "Book",
    "select": [
        "Book.id",
        "Book.title",
        "Book.isbn",
        "Author.first_name",
        "Author.last_name"
    ],
    "joins": [
        {"BookAuthor": "BookAuthor.book_id = Book.id"},
        {"Author": "Author.id = BookAuthor.author_id"}
    ],
    "order_by": ["Book.title"],
    "limit": 10
}

books_result = execute_query(books_query, format='records')
pretty_print(books_result)

### Convert Query Results to DataFrame

In [ ]:
# If we got data, convert to DataFrame for analysis
if 'data' in books_result and 'data' in books_result['data']:
    records = books_result['data']['data']
    df = pd.DataFrame(records)
    display(df)

## Using Generic Endpoint

Let's test the generic endpoint functionality to call any Coframe operation.

In [ ]:
def call_endpoint(operation, params=None):
    response = requests.post(f"{BASE_URL}/endpoint/{operation}", headers=headers, json=params or {})
    return response.json()

books = call_endpoint('books', {})
pretty_print(books)

## Red file from filesystem

Read some file from filesystem, only files in whitelisted directories are allowed

In [ ]:
def read_file(params=None):
    response = requests.post(f"{BASE_URL}/read_file", headers=headers, json=params or {})
    return response.json()

file = {
    "file_path": "data/book_list.yaml"
}
result = read_file(file)
pretty_print(result)

file = {
    "base_dir": "images",
    "file_path": "python_logo.png",
    "binary_encoding": "base64"
}
result = read_file(file)
pretty_print(result)

file = {
    "base_dir": "~/",
    "file_path": ".bashrc"
}
result = read_file(file)
pretty_print(result)



## User Profile Access

Let's test accessing the current user's profile.

In [ ]:
def get_profile():
    response = requests.get(f"{BASE_URL}/profile", headers=headers)
    return response.json()

my_profile = get_profile()
pretty_print(my_profile)

## Error Handling Test

Let's test how the API handles errors.

In [ ]:
# Try to get a non-existent item
non_existent = get_one('Book', 999999)
pretty_print(non_existent)

In [ ]:
# Try to use an invalid token
invalid_headers = {
    'Authorization': 'Bearer invalid_token',
    'Content-Type': 'application/json'
}

response = requests.get(f"{BASE_URL}/profile", headers=invalid_headers)
pretty_print(response.json())

## Batch Operations

Let's test how to perform batch operations.

In [ ]:
# Create multiple books in sequence
new_books = [
    {
        'title': 'Machine Learning Basics',
        'isbn': '9781234567891',
        'publication_date': '2025-01-15',
        'price': 39.99,
        'status': 'A'
    },
    {
        'title': 'Advanced Database Systems',
        'isbn': '9781234567892',
        'publication_date': '2025-02-20',
        'price': 45.99,
        'status': 'A'
    },
    {
        'title': 'Web Development with Flask',
        'isbn': '9781234567893',
        'publication_date': '2025-03-10',
        'price': 32.99,
        'status': 'A'
    }
]

created_books = []
for book in new_books:
    result = create('Book', book)
    if result.get('status') == 'success':
        created_books.append(result.get('data'))

print(f"Created {len(created_books)} books")
pretty_print(created_books)

## Filtering Records

Let's test filtering records using query parameters.

In [ ]:
# Get books with complex filtering using the query endpoint
filter_query = {
    "from": "Book",
    "select": ["id", "title", "price", "status"],
    "filters": {
        "conditions": [
            {"price": [">=", 35.0]},
            {"status": "A"}
        ]
    },
    "order_by": [["price", "desc"]],
    "limit": 10
}

filtered_books = execute_query(filter_query)
pretty_print(filtered_books)

## Performing Aggregations

Let's test performing aggregations with the query endpoint.

In [ ]:
# Aggregate query to get book statistics
agg_query = {
    "from": "Book",
    "select": [
        "count(id) as book_count",
        "avg(price) as avg_price",
        "min(price) as min_price",
        "max(price) as max_price",
        "sum(price) as total_price"
    ]
}

book_stats = execute_query(agg_query)
pretty_print(book_stats)

## Working with Relationships

Let's test working with relationships in the database.

In [ ]:
# Add an author
new_author = {
    'first_name': 'Jane',
    'last_name': 'Doe',
    'nationality': 'American',
    'birth_date': '1980-10-06',
}

author_result = create('Author', new_author)
pretty_print(author_result)

# If author creation was successful, create a book-author relationship
if 'data' in author_result and 'id' in author_result['data']:
    author_id = author_result['data']['id']

    # Get the first book we created
    if created_books and 'id' in created_books[0]:
        book_id = created_books[0]['id']

        # Create relationship
        book_author = {
            'book_id': book_id,
            'author_id': author_id,
            'notes': 'Created via API test'
        }

        relation_result = create('BookAuthor', book_author)
        pretty_print(relation_result)

## Cleanup

Let's clean up the data we created during testing.

In [ ]:
# Delete all the books we created
for book in created_books:
    if 'id' in book:
        result = delete('Book', book['id'])
        print(f"Deleted book {book['id']}: {result['status']}")

# Delete the author we created
if 'data' in author_result and 'id' in author_result['data']:
    result = delete('Author', author_result['data']['id'])
    print(f"Deleted author {author_result['data']['id']}: {result['status']}")